In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [2]:
pair  = "CAD_CHF"
granularity = "H1"
ma_list = [16, 64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [3]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
non_cols = ['time', 'volume']

In [5]:
mod_cols = [x for x in df.columns if x not in non_cols]

In [6]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [7]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [8]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2024-09-03T18:00:00.000000000Z,0.62789,0.62800,0.62765,0.62776
1,2024-09-03T19:00:00.000000000Z,0.62775,0.62813,0.62746,0.62756
2,2024-09-03T20:00:00.000000000Z,0.62757,0.62786,0.62748,0.62755
3,2024-09-03T21:00:00.000000000Z,0.62708,0.62766,0.62708,0.62754
4,2024-09-03T22:00:00.000000000Z,0.62750,0.62786,0.62750,0.62768


In [9]:
for ma in ma_list:
   df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [10]:
df_ma.dropna(inplace=True)

In [11]:
df_ma.reset_index(drop=True, inplace=True)

In [24]:

df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
0,2024-09-06T09:00:00.000000000Z,0.62284,0.62406,0.62274,0.62398,0.624633,0.626277,-0.001644,NaN,0
1,2024-09-06T10:00:00.000000000Z,0.62399,0.62450,0.62368,0.62410,0.624537,0.626220,-0.001682,-0.001644,0
2,2024-09-06T11:00:00.000000000Z,0.62410,0.62500,0.62397,0.62482,0.624508,0.626177,-0.001669,-0.001682,0
3,2024-09-06T12:00:00.000000000Z,0.62482,0.62784,0.62187,0.62508,0.624503,0.626138,-0.001635,-0.001669,0
4,2024-09-06T13:00:00.000000000Z,0.62502,0.62712,0.62444,0.62554,0.624527,0.626107,-0.001580,-0.001635,0


In [25]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

In [26]:
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)

In [27]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [28]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [29]:
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [32]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
31,2024-09-09T16:00:00.000000000Z,0.62539,0.62616,0.62532,0.62610,0.624697,0.624645,0.000052,-0.000186,1,-37.3,-37.3
57,2024-09-10T18:00:00.000000000Z,0.62224,0.62250,0.62190,0.62237,0.624129,0.624210,-0.000081,0.000104,-1,53.9,-53.9
83,2024-09-11T20:00:00.000000000Z,0.62814,0.62869,0.62776,0.62776,0.624331,0.624265,0.000066,-0.000363,1,-43.4,-43.4
122,2024-09-13T11:00:00.000000000Z,0.62460,0.62462,0.62276,0.62342,0.625568,0.625625,-0.000057,0.000181,-1,-10.8,10.8
185,2024-09-18T02:00:00.000000000Z,0.62216,0.62249,0.62200,0.62234,0.622576,0.622546,0.000031,-0.000060,1,-9.0,-9.0


In [30]:
df_trades['DELTA'] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)


df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [31]:
df_trades["GAIN"].sum()

np.float64(-813.3999999999996)

In [21]:
df_plot = df_ma.iloc[0:24].copy()

In [22]:
df_plot.shape

(24, 10)

In [23]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#24A06B", 
    decreasing_fillcolor="#CC2E3C", 
    increasing_line_color = "#2EC886", 
    decreasing_line_color = "#FF3A4C"

))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col], 
        line=dict(width=2), 
        line_shape='spline', 
        name=col))
fig.update_layout(width=1000, height=400, 
    margin=dict(l=10, r=10, t=10, b=10),
    font=dict(size=10, color="#e1e1e1"),
    paper_bgcolor="#1e1e1e", plot_bgcolor="#1e1e1e")
fig.update_xaxes(gridcolor="#1f292f", fixedrange=True, showgrid=True, rangeslider=dict(visible=False))
fig.update_yaxes(gridcolor="#1f292f", showgrid=True)


fig.show()